In [20]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [21]:
r.hget(b"rating", b"rating:User ID: 2, Movie ID: 136485")
r.keys("rating:User ID: 2, *")

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [37]:
r.hset('rating', key="rating:User ID: 2, Movie ID: 136485", value=3.1)

0

In [38]:
r.hgetall("rating:User ID: 2, Movie ID: 136485")

{}

In [39]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

True

In [40]:
r.get("Bahamas")

b'Nassau'

In [43]:
r.delete("Croatia")

1

In [44]:
r.get("Bahamas")

In [46]:
r.hset('rating', key="rating:User ID: 2, Movie ID: 136485", value="3.1")

0

In [48]:
import json
r.hset("rating:User ID: 2, Movie ID: 136485", )

True

In [50]:
r.get("rating:User ID: 2, Movie ID: 136485")

b'{"movieId": "136485", "userId": "2", "rating": "3.1"}'

In [56]:
r.delete("rating:User ID: 2, Movie ID: 136485")

1

In [54]:
r.hsetnx('rating:User ID: 2, Movie ID: 136485', key = "rating", value = 3.1)

1

In [57]:
r.hgetall('rating:User ID: 2, Movie ID: 136485')

{}

In [1]:
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, row_number
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from engine import RecommendationEngine
from pyspark.sql.window import Window

def init_spark_context():
    MAX_MEMORY = "5g"
    conf = SparkConf().setAll([("spark.app.name", "Spark_Processor"), ("spark.redis.port", "6379"), 
                                      ("spark.jars", "spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar"), 
                                      ("spark.executor.memory", MAX_MEMORY), ("spark.driver.memory", MAX_MEMORY), 
                                      ("spark.memory.fraction", "0.6")])
    sc = SparkContext(conf=conf)
    return sc

#global sc
sc = init_spark_context()
spark = SparkSession.builder.config(conf = sc.getConf()).getOrCreate()
#print(sc.getConf().getAll())
#print(read_moviedf.show())
#read_moviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").load()
#global moviedf
#moviedf = read_moviedf.sort("title")
datasets_path = os.path.join('..','movie_recommendation_system', 'datasets')
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
ratingschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("rating", DoubleType(), True)\
    .add("timeStamp", IntegerType(), True)

#global ratingdf
ratingdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(ratingschema)\
    .load(complete_ratings_file)

#global ratingdf
ratingdf = ratingdf.drop("timestamp")
#global ratingdf
ratingdf = ratingdf.withColumn("key", (concat(lit("User ID: "),col("userId"),lit(", Movie ID: "),col("movieID"))))
#global ratingdf
ratingdf = ratingdf.select("key", "userId", "movieId", "rating")
#global ratColumns
ratColumns = ratingdf.columns
#print(moviedf.show())
#print(ratingdf.show())
#newColumns = spark.createDataFrame([], ratingdf.schema)
#recEng = RecommendationEngine(sc, moviedf, ratingdf)

In [2]:
ratingdf.printSchema()

root
 |-- key: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [2]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')

In [7]:
meanRating = ratingdf.rdd.map(lambda x: x[3]).mean()


In [8]:
from operator import add
import math

baselineRmse = math.sqrt(ratingdf.rdd.map(lambda x: (meanRating - x[3]) ** 2).reduce(add) / ratingdf.count())
baselineRmse


1.06074393993063

In [19]:
b = [a[i + 1] - a[i] for i in range(len(a) - 1)]
print(max(a))

162541


In [4]:
newRow = spark.createDataFrame([("User ID: 79910, Movie ID: 7318", 79910, 7318, 0.6)], ratColumns)
newColumns = newColumns.union(newRow)
newColumns.show()

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 1, Movie...|     1|      1|   5.0|
|User ID: 1, Movie...|     1|      2|   5.0|
|User ID: 79910, M...| 79910|   7318|   0.6|
+--------------------+------+-------+------+



In [6]:
mergedDf = ratingdf.unionAll(newColumns)
mergedDf = mergedDf.withColumn("_row_number", row_number().over(Window.partitionBy(mergedDf['key']).orderBy(col('key').desc())))
mergedDf.show()

+--------------------+------+-------+------+-----------+
|                 key|userId|movieId|rating|_row_number|
+--------------------+------+-------+------+-----------+
|User ID: 1000, Mo...|  1000|   1380|   2.0|          1|
|User ID: 100001, ...|100001|    527|   4.0|          1|
|User ID: 100001, ...|100001|  72998|   4.5|          1|
|User ID: 100002, ...|100002|   2959|   2.5|          1|
|User ID: 100003, ...|100003|   1722|   3.0|          1|
|User ID: 100004, ...|100004|   1250|   5.0|          1|
|User ID: 100004, ...|100004|    306|   2.0|          1|
|User ID: 100004, ...|100004|    780|   2.0|          1|
|User ID: 100004, ...|100004|    903|   3.0|          1|
|User ID: 100007, ...|100007|  57669|   4.0|          1|
|User ID: 100007, ...|100007|  96811|   3.5|          1|
|User ID: 100009, ...|100009|   1884|   4.0|          1|
|User ID: 100009, ...|100009|   2401|   3.0|          1|
|User ID: 100009, ...|100009|   3030|   3.0|          1|
|User ID: 100009, ...|100009|  

In [30]:
#mergedDf.filter("_row_number > 1").orderBy('_row_number')
cutCols = newColumns.selectExpr("key as newKey", "rating as newRating")
print(cutCols.show())
full_outer_join = ratingdf.join(cutCols, ratingdf.key == cutCols.newKey ,how='full').select(col('key'), col('userId'), col('movieId'), col('rating'), col('newRating'))
full_outer_join.show()

+--------------------+---------+
|              newKey|newRating|
+--------------------+---------+
|User ID: 1, Movie...|      5.0|
|User ID: 1, Movie...|      5.0|
|User ID: 79910, M...|      5.0|
+--------------------+---------+

None
+--------------------+------+-------+------+---------+
|                 key|userId|movieId|rating|newRating|
+--------------------+------+-------+------+---------+
|User ID: 1000, Mo...|  1000|   1380|   2.0|     null|
|User ID: 100001, ...|100001|    527|   4.0|     null|
|User ID: 100001, ...|100001|  72998|   4.5|     null|
|User ID: 100002, ...|100002|   2959|   2.5|     null|
|User ID: 100003, ...|100003|   1722|   3.0|     null|
|User ID: 100004, ...|100004|   1250|   5.0|     null|
|User ID: 100004, ...|100004|    306|   2.0|     null|
|User ID: 100004, ...|100004|    780|   2.0|     null|
|User ID: 100004, ...|100004|    903|   3.0|     null|
|User ID: 100007, ...|100007|  57669|   4.0|     null|
|User ID: 100007, ...|100007|  96811|   3.5|    

In [32]:
full_outer_join.filter("newRating is not NULL").show()

+--------------------+------+-------+------+---------+
|                 key|userId|movieId|rating|newRating|
+--------------------+------+-------+------+---------+
|                null|  null|   null|  null|      5.0|
|                null|  null|   null|  null|      5.0|
|User ID: 79910, M...| 79910|   7318|   0.5|      5.0|
+--------------------+------+-------+------+---------+



In [22]:
newRatingList = updates.select('*').collect()
print(newRatingList)
newRating_key = [row['key'] for row in newRatingList]
newRating_rating = [float(row['rating']) for row in newRatingList]
print(newRating_key)
print(newRating_rating)

[Row(key='User ID: 79910, Movie ID: 7318', userId=79910, movieId=7318, rating=5.0)]
['User ID: 79910, Movie ID: 7318']
[5.0]


In [36]:
mergedDf.groupBy('key').count().show()

KeyboardInterrupt: 

In [11]:
counts = mergedDf.groupBy('key').count().selectExpr('key as newKey', 'count as count')
full_outer_join = mergedDf.join(counts, mergedDf.key == counts.newKey ,how='inner').select(col('key'), col('userId'), col('movieId'), col('rating'), col('_row_number'), col('count'))
full_outer_join.show()

+--------------------+------+-------+------+-----------+-----+
|                 key|userId|movieId|rating|_row_number|count|
+--------------------+------+-------+------+-----------+-----+
|User ID: 1000, Mo...|  1000|   1380|   2.0|          1|    1|
|User ID: 100001, ...|100001|    527|   4.0|          1|    1|
|User ID: 100001, ...|100001|  72998|   4.5|          1|    1|
|User ID: 100002, ...|100002|   2959|   2.5|          1|    1|
|User ID: 100003, ...|100003|   1722|   3.0|          1|    1|
|User ID: 100004, ...|100004|   1250|   5.0|          1|    1|
|User ID: 100004, ...|100004|    306|   2.0|          1|    1|
|User ID: 100004, ...|100004|    780|   2.0|          1|    1|
|User ID: 100004, ...|100004|    903|   3.0|          1|    1|
|User ID: 100007, ...|100007|  57669|   4.0|          1|    1|
|User ID: 100007, ...|100007|  96811|   3.5|          1|    1|
|User ID: 100009, ...|100009|   1884|   4.0|          1|    1|
|User ID: 100009, ...|100009|   2401|   3.0|          1

In [12]:
full_outer_join.filter('count > 1').show()

+--------------------+------+-------+------+-----------+-----+
|                 key|userId|movieId|rating|_row_number|count|
+--------------------+------+-------+------+-----------+-----+
|User ID: 79910, M...| 79910|   7318|   0.5|          1|    2|
|User ID: 79910, M...| 79910|   7318|   0.6|          2|    2|
+--------------------+------+-------+------+-----------+-----+



In [16]:
fixedDf = full_outer_join.filter('count == 1').unionAll(full_outer_join.filter((full_outer_join['count'] > 1) & (full_outer_join['_row_number'] > 1)))

In [18]:
fixedDf.filter(fixedDf['count'] > 1).show()

+--------------------+------+-------+------+-----------+-----+
|                 key|userId|movieId|rating|_row_number|count|
+--------------------+------+-------+------+-----------+-----+
|User ID: 79910, M...| 79910|   7318|   0.6|          2|    2|
+--------------------+------+-------+------+-----------+-----+



In [19]:
ratingdf = fixedDf.select(col('key'), col('userId'), col('movieId'), col('rating'))

In [20]:
ratingdf.filter("key == 'User ID: 1, Movie ID: 1'").show()

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 1, Movie...|     1|      1|   5.0|
+--------------------+------+-------+------+



In [24]:
import json
movies = [171595, 205277, 27313, 135777, 182307, 153536, 199748, 149484, 135492, 145196]
recMovieList = moviedf.filter(col('movieId').isin(movies)).collect()
#print(recMovieList)
a = []
keys = [row['title'] for row in recMovieList]
print(keys)
for key in keys:
    #newStr = key.decode('utf-8')
    movieId = r.hgetall('movie:' + key)[b'movieId'].decode("utf-8")
    #val = key('utf-8')[6:]
    val = key
    print(val)
    title = val[:-7]
    date = val[-5:-1]
    a.append((movieId, title, date))

print(json.dumps(a))

['Roberto Succo (2001)', 'Kaaka Muttai (2015)', 'Golmaal (2006)', "Mickey's Polo Team (1936)", 'All About My Wife (2012)', 'Long Pigs (2007)', 'Countdown (2004)', 'Funny Little Bunnies (1934)', 'The Drug King (2018)', 'Inside Out (1991)']
Roberto Succo (2001)
Kaaka Muttai (2015)
Golmaal (2006)
Mickey's Polo Team (1936)
All About My Wife (2012)
Long Pigs (2007)
Countdown (2004)
Funny Little Bunnies (1934)
The Drug King (2018)
Inside Out (1991)
[["27313", "Roberto Succo", "2001"], ["135492", "Kaaka Muttai", "2015"], ["135777", "Golmaal", "2006"], ["145196", "Mickey's Polo Team", "1936"], ["149484", "All About My Wife", "2012"], ["153536", "Long Pigs", "2007"], ["171595", "Countdown", "2004"], ["182307", "Funny Little Bunnies", "1934"], ["199748", "The Drug King", "2018"], ["205277", "Inside Out", "1991"]]


In [3]:
movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

movie = spark.read.option("inferSchema", "true")\
    .option("header", "true").csv(movies_file)

movie.show()
movie.printSchema()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [16]:
movieIds = list(movie.select(col('movieId')).toPandas()['movieId'])
ids = movieIds[20:50]
movieNames = list(movie.filter(movie.movieId.isin(ids)).toPandas()['title'])
movieNames

['Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Wings of Courage (1995)',
 'Babe (1995)',
 'Carrington (1995)',
 'Dead Man Walking (1995)',
 'Across the Sea of Time (1995)',
 'It Takes Two (1995)',
 'Clueless (1995)',
 'Cry, the Beloved Country (1995)',
 'Richard III (1995)',
 'Dead Presidents (1995)',
 'Restoration (1995)',
 'Mortal Kombat (1995)',
 'To Die For (1995)',
 'How to Make an American Quilt (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Pocahontas (1995)',
 'When Night Is Falling (1995)',
 'Usual Suspects, The (1995)']

In [3]:
#moviedf = movie.drop("genres")
#rint(moviedf.show())
movie.write.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").save()
#movie.filter(col('genres') == "Comedy").show()

In [4]:
(splitDf0, newVal) = ratingdf.randomSplit([0.1, 0.9])
(splitDf1, newVal) = ratingdf.randomSplit([0.11, 0.89])
(splitDf2, newVal) = ratingdf.randomSplit([0.125, 0.875])
(splitDf3, newVal) = ratingdf.randomSplit([0.143, 0.857])
(splitDf4, newVal) = ratingdf.randomSplit([0.167, 0.833])
(splitDf5, newVal) = ratingdf.randomSplit([0.2, 0.8])
(splitDf6, newVal) = ratingdf.randomSplit([0.25, 0.75])
(splitDf7, newVal) = ratingdf.randomSplit([0.33, 0.67])
(splitDf8, splitDf9) = ratingdf.randomSplit([0.5, 0.5])

splitDf0.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(0))
splitDf1.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(1))
splitDf2.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(2))
splitDf3.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(3))
splitDf4.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(4))
splitDf5.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(5))
splitDf6.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(6))
splitDf7.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(7))
splitDf8.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(8))
splitDf9.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()
print(str(9))

0


Py4JJavaError: An error occurred while calling o300.save.
: java.lang.IllegalStateException: SaveMode is set to ErrorIfExists and dataframe already exists in Redis and contains data.
	at org.apache.spark.sql.redis.DefaultSource.createRelation(DefaultSource.scala:27)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScop

In [7]:
ratingdf.toPandas().to_csv('datasets/newRatings.csv')

In [7]:
    read_moviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").load()
    moviedf = read_moviedf.sort("title")
    vals = r.keys("rating:User ID: 1, *")
    movies = []
    for st in vals:
        numbers = [int(word) for word in st.split() if word.isdigit()]
        movies.append(numbers[0])
    movieNames = moviedf.filter(moviedf.movieId.isin(movies)).select(col('title'))
    movieNames = [row['title'] for row in movieNames.collect()]
    print(movieNames)

['Delicatessen (1991)', 'Dolls (2002)', 'Naqoyqatsi (2002)', 'NeverEnding Story, The (1984)', 'Noi the Albino (Nói albinói) (2003)', 'Persona (1966)', "Spanish Apartment, The (L'auberge espagnole) (2002)", 'Three Colors: Red (Trois couleurs: Rouge) (1994)']
